# Tugas 1 MLOps: Membuat Machine Learning Pipeline

### Import Library

In [1]:
import tensorflow as tf  # Mengimpor library TensorFlow

from tfx.components import (
    CsvExampleGen,    # Menghasilkan contoh data dari file CSV
    StatisticsGen,    # Menghasilkan statistik data
    SchemaGen,        # Menghasilkan skema dari data
    ExampleValidator, # Memvalidasi contoh data berdasarkan skema
    Transform,        # Mentransformasi data untuk persiapan model
    Trainer,          # Melatih model machine learning
    Tuner             # Menyetel hyperparameter model
)

from tfx.proto import example_gen_pb2  # Mengimpor konfigurasi contoh data dari TFX

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext  # Mode interaktif untuk pengembangan dan debugging pipeline

import os  # Mengimpor modul untuk interaksi dengan sistem operasi

import pandas as pd  # Mengimpor library untuk analisis dan manipulasi data


### Set Variable

Kita menyatakan variabel yang akan digunakan untuk path pada directory yang akan digunakan untuk mengakses file yang dibutuhkan atau menyimpan file yang dihasilkan. Tujuannya adalah untuk mengatur lingkungan kerja dan penyimpanan yang dibutuhkan untuk melatih dan menyebarkan model pembelajaran mesin secara terstruktur dan efisien. Hal ini mencakup pemberian nama folder pipeline menyesuaikan kriteria pengumpulan proyek.

In [2]:
PIPELINE_NAME = "ivana27lita-pipeline"
SCHEMA_PIPELINE_NAME = "humour-tfdv-schema"

#Directory untuk menyimpan artifact yang akan dihasilkan
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

# from absl import logging
# logging.set_verbosity(logging.INFO)

### Import Dataset

Kita melakukan import dataset dalam bentuk csv yang telah diunduh melalui sumber pada link berikut: [200K SHORT TEXTS FOR HUMOR DETECTION](https://www.kaggle.com/datasets/deepcontractor/)

Kemudian kita membaca dataset dengan library pandas agar lebih memahami dataset.

Dataset terdiri dari 1 kolom fitur text dan 1 kolom target label yang mengindikasi apakah teks bersifat humoris (1) atau tidak (0). Terdapat 200k baris data pada dataset. 

In [3]:
# Mengambil dan membaca dataset
humour = pd.read_csv("data/dataset.csv")
humour.info()
humour.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    200000 non-null  object
 1   humor   200000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.1+ MB


,text,humor
0,"Joe biden rules out 2020 bid: 'guys, i'm not r...",0
1,Watch: darvish gave hitter whiplash with slow ...,0
2,What do you call a turtle without its shell? d...,1
3,5 reasons the 2016 election feels so personal,0
4,"Pasco police shot mexican migrant from behind,...",0


In [4]:
# variabel untuk mengakses sumber data, yaitu pada folder 'data' 
# dan object InteractiveContext() yang akan digunakan dalam projek.
DATA_ROOT = "data"
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

### Data Ingestion
Data ingestion - tahap awal dari proses pengolahan data yang terdapat pada machine learning pipeline. Pada tahap ini, kita memasukkan dan memproses data ke dalam format tertentu yang sesuai dengan format standar pada pipeline tersebut.

Penggunaan ExampleGen - komponen TFX yang digunakan untuk memecah data input menjadi beberapa set, seperti training & evaluation

Kita menggunakan komponen CsvExampleGen dalam TFX pipeline, yang berfungsi untuk membaca data dari file CSV dan membagi data tersebut menjadi bagian pelatihan (train) dan evaluasi (eval).

In [ ]:
# Pada ExampleGen di bawah akan membagi dataset untuk proses training dan evaluasi menjadi rasio 80:20

output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits = [
        example_gen_pb2.SplitConfig.Split(name = "train", hash_buckets = 8),
        example_gen_pb2.SplitConfig.Split(name = "eval", hash_buckets = 2)
    ])
)

example_gen = CsvExampleGen(input_base = DATA_ROOT, output_config = output)

Selanjutnya kita menjalankan komponen example_gen dalam konteks interaktif menggunakan TFX interactive_context

In [ ]:
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 23
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

### Data Validation
Data validation - kita melakukan proses validasi terhadap data yang baru dan memberi peringatan apabila terdapat anomali pada data tersebut.

Terdapat tiga komponen yang akan diperiksa pada proses data validation:
- Pemeriksaan parameter statistik. Pemeriksaan ini dilakukan untuk memastikan parameter statistik dari data yang baru sesuai dengan training dataset.
- Pemeriksaan data schema. Hal ini dilakukan untuk memastikan tidak terjadi perubahan dalam sebuah data schema.
- Pemeriksaan anomali. Pemeriksaan ini untuk mendeteksi apabila terjadi anomali pada suatu data. 

Kita akan menggunakan beberapa komponen yang disediakan oleh TFX, seperti StatisticGen, SchemaGen, dan ExampleValidator untuk proses data validation

Penggunaan **StatisticGen** - komponen TFX yang digunakan untuk menghitung statistik deskriptif dari dataset.
Komponen StatisticGen() pada TFX akan memasukkan parameter input 'examples' untuk menerima dataset dari komponen ExampleGen pada tahapan sebelumnya.

In [ ]:
statistic_gen = StatisticsGen(
    examples = example_gen.outputs["examples"]
)

interactive_context.run(statistic_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 24
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [ ]:
interactive_context.show(statistic_gen.outputs['statistics'])

Penggunaan **SchemaGen** - untuk membuat data schema
Komponen ini akan menerima input berupa summary statistic dari tahapan StatisticGen sebelumnya

In [ ]:
schema_gen = SchemaGen(statistics = statistic_gen.outputs["statistics"])

interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 25
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [ ]:
# Membuat SchemaGen untuk memperoleh informasi dari feature text dan label humoour
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'humor',INT,required,,-
'text',BYTES,required,,-


Terlihat dari SchemaGen yang telah dibuat, terdapat dua feature yaitu humour dan text, humour disini merupakan label untuk menentukan prediksi apakah text berupa kalimat yang menandakan humour atau tidak, kemudian text merupakan inputan untuk menentukan label, label bertipe integer dengan value 1 atau 0 dan text bertipe bytes dan masing-masing feature tersebut required untuk melakukan prediksi

Penggunaan **ExampleValidator** - untuk mendeteksi anomali pada data, menggunakan StatisticsGen dan SchemaGen yang telah dibuat sebelumnya

In [ ]:
example_validator = ExampleValidator(
    statistics = statistic_gen.outputs["statistics"],
    schema = schema_gen.outputs["schema"]
)

interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 26
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [ ]:
interactive_context.show(example_validator.outputs["anomalies"])

Terlihat bahwa data yang akan digunakan tidak terdapat anomali, sehingga dapat digunakan untuk proses pipeline selanjutnya

### Data Preprocessing

Tujuan dari tahap ini adalah mengubah data mentah menjadi data yang siap digunakan untuk melatih model

Penggunaan **Transform** - Transform akan menerima inputan berupa module file yang berisi preprocessing function.

In [ ]:
# Membuat file humour_transofrm.py untuk menyimpan proses transform pada variabel module humour_transform.py
TRANSFORM_MODULE_FILE = "humour_transform.py"

In [ ]:
%%writefile {TRANSFORM_MODULE_FILE}

# Mengimpor TensorFlow dan modul yang diperlukan dari TensorFlow Transform (TFX)
import tensorflow as tf
from tensorflow_transform.tf_metadata import schema_utils
import tensorflow_transform as tft
import tensorflow_transform.beam as tft_beam

# Mendefinisikan nama kunci untuk label (humor) dan fitur (text) dalam dataset.
LABEL_KEY = "humor"
FEATURE_KEY = "text"

# Fungsi untuk menghasilkan nama kunci fitur yang telah ditransformasi dengan menambahkan suffix "_xf".
def transformed_name(key):
    return key + "_xf"

# Fungsi preprocessing - menyiapkan transformasi yang diperlukan untuk data, seperti mengubah teks menjadi 
# huruf kecil dan memastikan label berada dalam format yang tepat
def preprocessing_fn(inputs):
    outputs = {}
    outputs[transformed_name(FEATURE_KEY)] = tf.strings.lower(inputs[FEATURE_KEY])
    outputs[transformed_name(LABEL_KEY)] = tf.cast(inputs[LABEL_KEY], tf.int64)
    return outputs


Overwriting humour_transform.py


In [ ]:
# Membuat transform yang berisi ExampleGen, SchemaGen dan module transform humour_transform.py
transform = Transform(
    examples = example_gen.outputs["examples"],
    schema = schema_gen.outputs['schema'],
    module_file = os.path.abspath(TRANSFORM_MODULE_FILE)
)

interactive_context.run(transform)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: pipelines\ivana27lita-pipeline\Transform\transform_graph\27\.temp_path\tftransform_tmp\e7c006ea21744148a486e9cc55393a1b\assets


INFO:tensorflow:Assets written to: pipelines\ivana27lita-pipeline\Transform\transform_graph\27\.temp_path\tftransform_tmp\e7c006ea21744148a486e9cc55393a1b\assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 27
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

### Model Development

Untuk menjalankan proses pengembangan dan validasi data, TFX menyediakan beberapa komponen seperti berikut.

- Komponen Trainer dan Tuner. Kedua komponen ini digunakan untuk menjalankan proses pengembangan model. 
- Komponen Evaluator dan Resolver. Kita dapat memanfaatkan kedua komponen ini untuk menganalisis dan memvalidasi model. Konfigurasi untuk melakukan proses ini didefinisikan menggunakan library TensorFlow Model Analysis (TFMA).

Penggunaan **Tuner** - Melakukan proses tuning hyperparameter secara otomatis pada arsitektur model yang digunakan untuk mendapatkan hyperparameter terbaik dengan bantuan keras tuner

In [ ]:
TUNER_MODULE_FILE = "humour_tuner.py"

In [ ]:
%%writefile {TUNER_MODULE_FILE}

import kerastuner as kt
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_transform as tft
from tfx.components.tuner.component import TunerFnResult
import os

LABEL_KEY = "humor"
FEATURE_KEY = "text"

VOCAB_SIZE = 10000
SEQUENCE_LENGTH = 100
NUM_EPOCHS = 5

vectorize_layer = layers.TextVectorization(
    standardize="lower_and_strip_punctuation",
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH
)

def transformed_name(key):
    return key + "_xf"

def gzip_reader_fn(filenames):
    return tf.data.TFRecordDataset(filenames, compression_type="GZIP")

def input_fn(file_pattern, tf_transform_output, num_epochs, batch_size=64):
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy()
    )
    
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=transformed_name(LABEL_KEY),
    )
    
    return dataset

def build_model(hp):
    embedding_dim = 16

    hp_units_1 = hp.Int('units_1', min_value=32, max_value=128, step=32)
    hp_units_2 = hp.Int('units_2', min_value=16, max_value=64, step=16)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3])

    inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)
    reshaped_text = tf.reshape(inputs, [-1])
    x = vectorize_layer(reshaped_text)
    x = layers.Embedding(VOCAB_SIZE, embedding_dim, name="embedding")(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(hp_units_1, activation='relu')(x)
    x = layers.Dense(hp_units_2, activation="relu")(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
        loss='binary_crossentropy',
        optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

    return model

def tuner_fn(fn_args):
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    train_dataset = input_fn(fn_args.train_files[0], tf_transform_output, NUM_EPOCHS)
    eval_dataset = input_fn(fn_args.eval_files[0], tf_transform_output, NUM_EPOCHS)
    
    # Adapt the vectorize_layer with the training dataset
    vectorize_layer.adapt(train_dataset.map(lambda x, y: x[transformed_name(FEATURE_KEY)]))
    
    early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

    tuner = kt.Hyperband(
        build_model,
        objective='val_binary_accuracy',
        max_epochs=10,
        factor=3,
        directory=fn_args.working_dir,
        project_name='kt_hyperband'
    )

    return TunerFnResult(
        tuner=tuner,
        fit_kwargs={
            "callbacks": [early_stopping_callback],
            'x': train_dataset,
            'validation_data': eval_dataset,
            'steps_per_epoch': fn_args.train_steps,
            'validation_steps': fn_args.eval_steps
        }
    )

Overwriting humour_tuner.py


In [ ]:
from tfx.components.tuner.component import Tuner
from tfx.proto import trainer_pb2
from tfx.proto import tuner_pb2

# Define the tuner component
tuner = Tuner(
    module_file=os.path.abspath(TUNER_MODULE_FILE),
    examples=transform.outputs["transformed_examples"],
    transform_graph=transform.outputs["transform_graph"],
    schema=schema_gen.outputs["schema"],
    train_args=trainer_pb2.TrainArgs(splits=["train"], num_steps=800),
    eval_args=trainer_pb2.EvalArgs(splits=["eval"], num_steps=400),
)

interactive_context.run(tuner)

Trial 30 Complete [00h 01m 08s]
val_binary_accuracy: 0.9419921636581421

Best val_binary_accuracy So Far: 0.9419921636581421
Total elapsed time: 00h 16m 06s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Results summary
Results in pipelines\ivana27lita-pipeline\.temp\28\kt_hyperband
Showing 10 best trials
Trial summary
Hyperparameters:
units_1: 64
units_2: 16
learning_rate: 0.01
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.9419921636581421
Trial summary
Hyperparameters:
units_1: 32
units_2: 48
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.94140625
Trial summary
Hyperparameters:
units_1: 32
units_2: 64
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0019
Score: 0.9412890672683716
Trial summary
Hyperparameters:
units_1: 128
units_2: 32
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.941210925579071
Trial summary
Hyperparameters:
units_1: 96
units_2: 32
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0015
Score: 0.94093751907

ExecutionResult(
    component_id: Tuner
    execution_id: 28
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={})
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}))


Penggunaan **Trainer** - terdapat beberapa tahapan yang akan dimuat dalam proses training menggunakan komponen Trainer diantaranya
- Memuat training dan validation dataset yang telah di-preprocessing.
- Memuat arsitektur model.
- Melatih model dengan training dataset serta memvalidasi model tersebut menggunakan validation dataset.
- Model yang telah dilatih selanjutnya akan diekspor agar dapat dievaluasi.

In [ ]:
#  Membuat file humour_trainer.py untuk menyimpan proses training pada variabel module humour_trainer.py
TRAINER_MODULE_FILE = "humour_trainer.py"

In [37]:
%%writefile {TRAINER_MODULE_FILE}
import os
import tensorflow as tf
import tensorflow_transform as tft
from tensorflow.keras import layers
from tfx.components.trainer.fn_args_utils import FnArgs

LABEL_KEY = 'humor'
FEATURE_KEY   = 'text'

def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def input_fn(file_pattern, tf_transform_output, num_epochs, batch_size=64) -> tf.data.Dataset:
    """Get post_transform feature & create batches of data"""

    # Get post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy()
    )

    # Create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern = file_pattern,
        batch_size   = batch_size,
        features     = transform_feature_spec,
        reader       = gzip_reader_fn,
        num_epochs   = num_epochs,
        label_key    = transformed_name(LABEL_KEY)
    )

    return dataset

# Vocabulary size and number of words in a sequence
VOCAB_SIZE      = 10000
SEQUENCE_LENGTH = 100

vectorize_layer = layers.TextVectorization(
    standardize            = 'lower_and_strip_punctuation',
    max_tokens             = VOCAB_SIZE,
    output_mode            = 'int',
    output_sequence_length = SEQUENCE_LENGTH
)

def model_builder(hp):
    """Build machine learning model"""
    inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)

    reshaped_narrative = tf.reshape(inputs, [-1])
    x = vectorize_layer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, hp.get('embedding_dim', 8), name='embedding')(x)
    x = layers.Bidirectional(layers.LSTM(hp.get('lstm_units', 16)))(x)
    for _ in range(hp.get('num_layers', 1)):
        x = layers.Dense(hp.get('dense_units', 32), activation='relu')(x)
    x = layers.Dropout(hp.get('dropout_rate', 0.5))(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs = inputs, outputs = outputs)
    model.compile(
        loss      = tf.keras.losses.BinaryCrossentropy(from_logits=True),
        optimizer = tf.keras.optimizers.Adam(hp.get('learning_rate', 0.001)),
        metrics   = [tf.keras.metrics.BinaryAccuracy()]
    )

    model.summary()
    return model

def _get_serve_tf_examples_fn(model, tf_transform_output):
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL_KEY)

        parsed_features      = tf.io.parse_example(serialized_tf_examples, feature_spec)
        transformed_features = model.tft_layer(parsed_features)

        # get predictions using the transformed features
        return model(transformed_features)

    return serve_tf_examples_fn

def run_fn(fn_args: FnArgs) -> None:
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')
    hp      = fn_args.hyperparameters['values']

    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir = log_dir, update_freq='batch'
    )

    early_stop_callback = tf.keras.callbacks.EarlyStopping(
        monitor  = 'val_binary_accuracy',
        mode     = 'max',
        verbose  = 1,
        patience = 10
    )

    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        fn_args.serving_model_dir,
        monitor        = 'val_binary_accuracy',
        mode           = 'max',
        verbose        = 1,
        save_best_only = True
    )

    callbacks = [
        tensorboard_callback,
        early_stop_callback,
        model_checkpoint_callback
    ]

    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    # Create batches of data
    train_set = input_fn(fn_args.train_files, tf_transform_output, num_epochs=hp.get('epochs', 10))
    val_set   = input_fn(fn_args.eval_files,  tf_transform_output, num_epochs=hp.get('epochs', 10))

    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)]
                for i in list(train_set)
        ]]
    )

    # Build the model
    model = model_builder(hp)

    # Train the model
    model.fit(
        x                = train_set,
        validation_data  = val_set,
        callbacks        = callbacks,
        steps_per_epoch  = fn_args.train_steps,
        validation_steps = fn_args.eval_steps,
        epochs           = hp.get('epochs', 7)
    )

    signatures = {
        'serving_default': _get_serve_tf_examples_fn(
            model, tf_transform_output
        ).get_concrete_function(
            tf.TensorSpec(
                shape = [None],
                dtype = tf.string,
                name  = 'examples'
            )
        )
    }

    model.save(
        fn_args.serving_model_dir,
        save_format = 'tf',
        signatures  = signatures
    )

Overwriting humour_trainer.py


Kode di bawah digunakan untuk mengkonfigurasi dan menjalankan komponen Trainer dalam pipeline TFX.

In [38]:
trainer = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples=transform.outputs["transformed_examples"],
    transform_graph=transform.outputs["transform_graph"],
    schema=schema_gen.outputs["schema"],
    hyperparameters=tuner.outputs['best_hyperparameters'],
    train_args=trainer_pb2.TrainArgs(splits=["train"], num_steps=800),
    eval_args=trainer_pb2.EvalArgs(splits=["eval"], num_steps=400),
)
interactive_context.run(trainer)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_xf (InputLayer)        [(None, 1)]               0         
                                                                 
 tf.reshape_1 (TFOpLambda)   (None,)                   0         
                                                                 
 text_vectorization_1 (TextV  (None, 100)              0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 100, 8)            80000     
                                                                 
 bidirectional (Bidirectiona  (None, 32)               3200      
 l)                                                              
                                                                 
 dense_3 (Dense)             (None, 32)                1056

d:\AdvancedBangkit\.venv\lib\site-packages\keras\backend.py:5673: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


800/800 [==============================] - ETA: 0s - loss: 0.2159 - binary_accuracy: 0.9179
Epoch 1: val_binary_accuracy improved from -inf to 0.93703, saving model to pipelines\ivana27lita-pipeline\Trainer\model\33\Format-Serving


INFO:tensorflow:Assets written to: pipelines\ivana27lita-pipeline\Trainer\model\33\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\ivana27lita-pipeline\Trainer\model\33\Format-Serving\assets


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: pipelines\ivana27lita-pipeline\Trainer\model\33\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\ivana27lita-pipeline\Trainer\model\33\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 33
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

Penggunaan **Resolver** - untuk menyediakan baseline model yang merupakan hal penting ketika kita akan memiliki lebih dari satu versi model dan ingin membandingkan dua versi model yang berbeda

In [40]:
from tfx.dsl.components.common.resolver import Resolver 
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy 
from tfx.types import Channel 
from tfx.types.standard_artifacts import Model, ModelBlessing 
 
model_resolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')
 
interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 35
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

Penggunaan **Evaluator**

Setelah mendefinisikan komponen Resolver, tahap selanjutnya adalah membuat beberapa konfigurasi untuk mengevaluasi model menggunakan komponen Evaluator.

Komponen Evaluator() akan menerima beberapa input, seperti examples, model, baseline_model, dan eval_config.

In [41]:
import tensorflow_model_analysis as tfma

eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='humor')],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            
            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name='FalsePositives'),
            tfma.MetricConfig(class_name='TruePositives'),
            tfma.MetricConfig(class_name='FalseNegatives'),
            tfma.MetricConfig(class_name='TrueNegatives'),
            tfma.MetricConfig(class_name='BinaryAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value':0.5}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value':0.0001})
                    )
            )
        ])
    ]
 
)

In [42]:
from tfx.components import Evaluator

evaluator = Evaluator(
    examples = example_gen.outputs['examples'],
    model = trainer.outputs['model'],
    baseline_model = model_resolver.outputs['model'],
    eval_config = eval_config
)

interactive_context.run(evaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 36
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

Kode di bawah akan menampilkan tampilan interaktif untuk hasil evaluasi dari komponen Evaluator() dengan menggunakan library TFMA.

In [43]:
# Visualize the evaluation results
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'binary_accu…

### Model Deployment

Penggunaan **Pusher** - untuk mengirim model yang telah divalidasi ke dalam target sistem deployment, baik dalam tahap training maupun proses retraining. 

In [44]:
from tfx.components import Pusher 
from tfx.proto import pusher_pb2 
 
pusher = Pusher(
model=trainer.outputs['model'],
model_blessing=evaluator.outputs['blessing'],
push_destination=pusher_pb2.PushDestination(
    filesystem=pusher_pb2.PushDestination.Filesystem(
        base_directory='serving_model_dir/humor-detection-model'))
 
)
 
interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 37
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))